In [13]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [16]:
# My custom Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 3) #26x26
        self.batNorm1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3) #24x24
        self.batNorm2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) # 12x12x16
        
        self.conv3 = nn.Conv2d(16, 32, 3) #10x10
        self.batNorm3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1, stride=2) #5x5
        self.batNorm4 = nn.BatchNorm2d(32)
        self.drop2 = nn.Dropout(0.15)
        
        self.conv5 = nn.Conv2d(32, 32, 3,stride=2) #3x3
        self.batNorm5 = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32, 16, 1)
        self.batNorm6 = nn.BatchNorm2d(16)
        # self.pool2 = nn.MaxPool2d(2, 2) #7x7x32
        self.drop3 = nn.Dropout(0.15)
        
        # self.conv5 = nn.Conv2d(32, 32, 3) # 5x5
        # self.batNorm5 = nn.BatchNorm2d(32)
        # self.conv6 = nn.Conv2d(32, 32, 3) #3x3
        # self.batNorm6 = nn.BatchNorm2d(32)
        # self.conv7 = nn.Conv2d(32, 32, 3) #1x1x32
        # self.batNorm7 = nn.BatchNorm2d(32)
        # self.drop3 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(256,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batNorm1(x)
        x = F.relu(self.conv2(x))
        x = self.batNorm2(x)
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = self.batNorm3(x)
        # x = F.relu(self.conv4(x))
        # x = self.batNorm4(x)
        x = self.drop2(x)
        
        x = F.relu(self.conv5(x))
        x = self.batNorm5(x)
        x = F.relu(self.conv6(x))
        x = self.batNorm6(x)
        
        x = self.drop3(x)
        
        
        # x = F.relu(self.conv6(x))
        # x = self.batNorm6(x)
        # x = F.relu(self.conv7(x))
        # x = self.batNorm7(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        # x = self.conv2(x)
        # x = self.pool1(F.relu(self.conv2(F.relu())))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        
        
        return F.log_softmax(x)

In [17]:
!pip install torchsummary
from torchsummary import summary

# run summary on cpu first then shift to device
model = Net().to("cpu")
summary(model, input_size=(1, 28, 28))

if torch.backends.mps.is_available():
    # Apple Silicon GPU
    device = "mps"
    use_cuda = True
elif torch.cuda.is_available():
    # nVidia GPU
    device = "cuda"
    use_cuda = True
else:
    # CPU
    device = "cpu"
print("Running models on: ", device)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,320
       BatchNorm2d-4           [-1, 16, 24, 24]              32
         MaxPool2d-5           [-1, 16, 12, 12]               0
            Conv2d-6           [-1, 32, 10, 10]           4,640
       BatchNorm2d-7           [-1, 32, 10, 10]              64
           Dropout-8           [-1, 32, 10, 10]               0
            Conv2d-9             [-1, 32, 4, 4]           9,248
      BatchNorm2d-10             [-1, 32, 4, 4]              64
           Conv2d-11             [-1, 16, 4, 4]             528
      BatchNorm2d-12             [-1, 16, 4, 4]              32
          Dropout-13             [-1, 16, 4, 4]               0
           Linear-14                   

/var/folders/6x/skj50cf15297r_q8sp6r6rwr0000gn/T/ipykernel_63965/1895879961.py:68: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [18]:


torch.manual_seed(1)
batch_size = 200

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [19]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [20]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/300 [00:00<?, ?it/s]

/var/folders/6x/skj50cf15297r_q8sp6r6rwr0000gn/T/ipykernel_63965/1895879961.py:68: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.11418912559747696 batch_id=299: 100%|██████████| 300/300 [00:10<00:00, 29.42it/s] 



Test set: Average loss: 0.0584, Accuracy: 9830/10000 (98%)



loss=0.03215485066175461 batch_id=299: 100%|██████████| 300/300 [00:06<00:00, 49.61it/s] 



Test set: Average loss: 0.0420, Accuracy: 9879/10000 (99%)



...


Test set: Average loss: 0.0222, Accuracy: 9925/10000 (99%)



loss=0.01799648068845272 batch_id=299: 100%|██████████| 300/300 [00:05<00:00, 51.56it/s]  



Test set: Average loss: 0.0213, Accuracy: 9934/10000 (99%)



loss=0.016354501247406006 batch_id=299: 100%|██████████| 300/300 [00:05<00:00, 52.49it/s] 



Test set: Average loss: 0.0221, Accuracy: 9927/10000 (99%)

